# Transformation Explorations

## Whole Code

### Version 1

In [0]:
# ====================================================
# ACCOUNTS PAYABLE INVOICES TRANSFORMATION PIPELINE
# ====================================================
"""
This script performs data transformation on the 'ap.bronze.ap_invoices' table.

Steps:
    1. Standardize date formats
    2. Clean text and ID fields
    3. Create an enriched dataset with calculated fields:
        - POAmount
        - OnTimePayment
        - AgingDays
        - Month

Assumptions:
    - 'today' is the latest date found in 'InvoiceDate' or 'PaidDate'.
    - If 'PaidDate' is blank, the invoice is considered unpaid.
    - If unpaid and 'DueDate' > 'today', the invoice is not considered overdue.
"""

# --- DEPENDENCIES ---
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType, IntegerType
from pyspark.sql.window import Window


# --- CONFIG ---
INPUT_TABLE = "ap.bronze.ap_invoices"
OUTPUT_TABLE = "ap.silver.ap_invoices_enriched"  # Adjust as needed
DATE_COLS = ["InvoiceDate", "DueDate", "PaidDate"]
TEXT_FIELDS = ["Supplier", "Category"]
ID_FIELDS = ["InvoiceID", "CostCenter", "POID", "Currency"]


# --- FUNCTIONS ---
def standardize_dates(df: DataFrame, columns: list) -> DataFrame:
    """
    Convert date columns to date type safely.
    Tries multiple formats before falling back to default parsing.
    """
    common_formats = [
        "MM/dd/yyyy",  # 10/30/2025
        "MM/dd/yy",    # 10/30/25
        "dd/MM/yyyy",  # 30/10/2025
        "yyyy/MM/dd",  # 2025/10/30
        "yyyy-MM-dd"   # 2025-10-30
    ]
    
    for col in columns:
        # Try each format
        date_col = None
        for fmt in common_formats:
            temp_col = F.to_date(F.col(col), fmt)
            if date_col is None:
                date_col = temp_col
            else:
                # Coalesce to try next format if previous failed
                date_col = F.coalesce(date_col, temp_col)
        
        # Replace original column with parsed date
        df = df.withColumn(col, date_col)
    
    return df


def clean_text_fields(df: DataFrame, text_cols: list, id_cols: list) -> DataFrame:
    """Clean text and ID fields: trim spaces, normalize casing."""
    
    # Clean text fields (trim and title case)
    for col in text_cols:
        df = df.withColumn(
            col,
            F.initcap(F.trim(F.col(col)))
        )
    
    # Clean ID fields (trim and upper case)
    for col in id_cols:
        df = df.withColumn(
            col,
            F.upper(F.trim(F.col(col)))
        )
    
    return df


def enrich_data(df: DataFrame) -> DataFrame:
    """Add calculated columns: POAmount, OnTimePayment, AgingDays, Month."""
    
    # Compute POAmount
    df = df.withColumn("POAmount", F.col("Quantity") * F.col("UnitPrice_PO"))
    
    # Define 'today' as the latest date from InvoiceDate or PaidDate
    today = df.select(
        F.greatest(
            F.max("InvoiceDate"),
            F.max("PaidDate")
        ).alias("today")
    ).first()["today"]
    
    # Create OnTimePayment column
    df = df.withColumn(
        "OnTimePayment",
        F.when(
            F.col("PaidDate").isNotNull() & (F.col("PaidDate") <= F.col("DueDate")),
            F.lit(True)
        ).when(
            F.col("PaidDate").isNotNull() & (F.col("PaidDate") > F.col("DueDate")),
            F.lit(False)
        ).when(
            F.col("PaidDate").isNull() & (F.col("DueDate") <= F.lit(today)),
            F.lit(False)
        ).otherwise(F.lit(None)).cast(BooleanType())
    )
    
    # Create AgingDays column
    df = df.withColumn(
        "AgingDays",
        F.when(
            F.col("PaidDate").isNull(),
            F.datediff(F.lit(today), F.col("DueDate"))
        ).otherwise(
            F.datediff(F.col("PaidDate"), F.col("InvoiceDate"))
        ).cast(IntegerType())
    )
    
    # Create Month column
    df = df.withColumn(
        "Month",
        F.date_format(F.col("InvoiceDate"), "yyyy-MM")
    )
    
    return df


# --- MAIN PIPELINE ---
def main():
    """Execute the AP Invoices data transformation pipeline."""
    
    # Get Spark session (already available in Databricks)
    spark = SparkSession.builder.getOrCreate()
    
    print(f"Loading source data from {INPUT_TABLE}...")
    df = spark.table(INPUT_TABLE)
    
    print("Standardizing date formats...")
    df = standardize_dates(df, DATE_COLS)
    
    print("Cleaning text and ID fields...")
    df = clean_text_fields(df, TEXT_FIELDS, ID_FIELDS)
    
    print("Enriching data with calculated columns...")
    df = enrich_data(df)
    
    print(f"Writing enriched data to {OUTPUT_TABLE}...")
    df.write.format("delta").mode("overwrite").saveAsTable(OUTPUT_TABLE)
    
    print("Transformation pipeline completed successfully!")
    
    # Return the DataFrame for further use if needed
    return df


# --- RUN SCRIPT ---
if __name__ == "__main__":
    result_df = main()
    # Optionally display results in Databricks notebook
    display(result_df)

### Version 2

In [0]:
"""
Silver Layer - Accounts Payable Invoices Transformation Pipeline
======================================================================
This script performs data transformation on the 'ap.bronze.ap_invoices' table.

Steps:
    1. Standardize date formats
    2. Clean text and ID fields
    3. Create an enriched dataset with calculated fields:
        - POAmount
        - OnTimePayment
        - AgingDays
        - Month

Assumptions:
    - 'today' is the latest date found in 'InvoiceDate' or 'PaidDate'.
    - If 'PaidDate' is blank, the invoice is considered unpaid.
    - If unpaid and 'DueDate' > 'today', the invoice is not considered overdue.
"""

# ============================================================================
# DEPENDENCIES
# ============================================================================
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType, IntegerType
from pyspark.sql.window import Window


# ============================================================================
# CONFIGURATION
# ============================================================================
INPUT_TABLE = "ap.bronze.ap_invoices"
OUTPUT_TABLE = "ap.silver.ap_invoices_enriched"  # Adjust as needed
DATE_COLS = ["InvoiceDate", "DueDate", "PaidDate"]
TEXT_FIELDS = ["Supplier", "Category"]
ID_FIELDS = ["InvoiceID", "CostCenter", "POID", "Currency"]


# ============================================================================
# FUNCTIONS
# ============================================================================
def standardize_dates(df: DataFrame, columns: list) -> DataFrame:
    """
    Convert date columns to date type safely.
    Tries multiple formats before falling back to default parsing.
    """
    common_formats = [
        "MM/dd/yyyy",  # 10/30/2025
        "MM/dd/yy",    # 10/30/25
        "dd/MM/yyyy",  # 30/10/2025
        "yyyy/MM/dd",  # 2025/10/30
        "yyyy-MM-dd"   # 2025-10-30
    ]
    
    for col in columns:
        # Try each format
        date_col = None
        for fmt in common_formats:
            temp_col = to_date(col(col), fmt)
            if date_col is None:
                date_col = temp_col
            else:
                # Coalesce to try next format if previous failed
                date_col = coalesce(date_col, temp_col)
        
        # Replace original column with parsed date
        df = df.withColumn(col, date_col)
    
    return df


def clean_text_fields(df: DataFrame, text_cols: list, id_cols: list) -> DataFrame:
    """Clean text and ID fields: trim spaces, normalize casing."""
    
    # Clean text fields (trim and title case)
    for col in text_cols:
        df = df.withColumn(
            col,
            initcap(trim(col(col)))
        )
    
    # Clean ID fields (trim and upper case)
    for col in id_cols:
        df = df.withColumn(
            col,
            upper(trim(col(col)))
        )
    
    return df


def enrich_data(df: DataFrame) -> DataFrame:
    """Add calculated columns: POAmount, OnTimePayment, AgingDays, Month."""
    
    # Compute POAmount
    df = df.withColumn("POAmount", col("Quantity") * col("UnitPrice_PO"))
    
    # Define 'today' as the latest date from InvoiceDate or PaidDate
    today = df.select(
        greatest(
            max("InvoiceDate"),
            max("PaidDate")
        ).alias("today")
    ).first()["today"]
    
    # Create OnTimePayment column
    df = df.withColumn(
        "OnTimePayment",
        when(
            col("PaidDate").isNotNull() & (col("PaidDate") <= col("DueDate")),
            lit(True)
        ).when(
            col("PaidDate").isNotNull() & (col("PaidDate") > col("DueDate")),
            lit(False)
        ).when(
            col("PaidDate").isNull() & (col("DueDate") <= lit(today)),
            lit(False)
        ).otherwise(lit(None)).cast(BooleanType())
    )
    
    # Create AgingDays column
    df = df.withColumn(
        "AgingDays",
        when(
            col("PaidDate").isNull(),
            datediff(lit(today), col("DueDate"))
        ).otherwise(
            datediff(col("PaidDate"), col("InvoiceDate"))
        ).cast(IntegerType())
    )
    
    # Create Month column
    df = df.withColumn(
        "Month",
        date_format(col("InvoiceDate"), "yyyy-MM")
    )
    
    return df


# ============================================================================
# TABLE CREATION
# ============================================================================
# Execute the AP Invoices data transformation pipeline.

print(f"Loading source data from {INPUT_TABLE}...")
df = spark.table(INPUT_TABLE)

print("Standardizing date formats...")
df = standardize_dates(df, DATE_COLS)

print("Cleaning text and ID fields...")
df = clean_text_fields(df, TEXT_FIELDS, ID_FIELDS)

print("Enriching data with calculated columns...")
df = enrich_data(df)

print(f"Writing enriched data to {OUTPUT_TABLE}...")
df.write.format("delta").mode("overwrite").saveAsTable(OUTPUT_TABLE)

print("Transformation pipeline completed successfully!")


# Display results
display(df)

## Por partes

Cambiar nombres de InvoiceDate a invoice_date 

In [0]:
"""
Silver Layer - Accounts Payable Invoices Transformation Pipeline
======================================================================
This script performs data transformation on the 'ap.bronze.ap_invoices' table.

Steps:
    1. Standardize date formats
    2. Clean text and ID fields
    3. Create an enriched dataset with calculated fields:
        - POAmount
        - OnTimePayment
        - AgingDays
        - Month

Assumptions:
    - 'today' is the latest date found in 'invoice_date' or 'paid_date'.
    - If 'paid_date' is blank, the invoice is considered unpaid.
    - If unpaid and 'due_date' > 'today', the invoice is not considered overdue.
"""

# ============================================================================
# DEPENDENCIES
# ============================================================================
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType, IntegerType
from pyspark.sql.window import Window


# ============================================================================
# CONFIGURATION
# ============================================================================
INPUT_TABLE = "ap.bronze.ap_invoices"
OUTPUT_TABLE = "ap.silver.ap_invoices_enriched"  # Adjust as needed
DATE_COLS = ["invoice_date", "due_date", "paid_date"]
TEXT_FIELDS = ["supplier", "category"]
ID_FIELDS = ["invoice_id", "cost_center", "poid"]


# ============================================================================
# FUNCTIONS
# ============================================================================
def standardize_dates(df: DataFrame, columns: list) -> DataFrame:
    """
    Convert date columns to date type safely.
    Tries multiple formats before falling back to default parsing.
    """
    common_formats = [
        "MM/dd/yyyy",  # 10/30/2025
        "MM/dd/yy",    # 10/30/25
        "dd/MM/yyyy",  # 30/10/2025
        "yyyy/MM/dd",  # 2025/10/30
        "yyyy-MM-dd"   # 2025-10-30
    ]
    
    for col in columns:
        # Try each format
        date_col = None
        for fmt in common_formats:
            temp_col = to_date(col(col), fmt)
            if date_col is None:
                date_col = temp_col
            else:
                # Coalesce to try next format if previous failed
                date_col = coalesce(date_col, temp_col)
        
        # Replace original column with parsed date
        df = df.withColumn(col, date_col)
    
    return df


def clean_text_fields(df: DataFrame, text_cols: list, id_cols: list) -> DataFrame:
    """Clean text and ID fields: trim spaces, normalize casing."""
    
    # Clean text fields (trim and title case)
    for col in text_cols:
        df = df.withColumn(
            col,
            initcap(trim(col(col)))
        )
    
    # Clean ID fields (trim and upper case)
    for col in id_cols:
        df = df.withColumn(
            col,
            upper(trim(col(col)))
        )
    
    return df


def enrich_data(df: DataFrame) -> DataFrame:
    """Add calculated columns: po_amount, on_time_payment, aging_days, month."""
    
    # Compute POAmount
    df = df.withColumn("po_amount", col("quantity") * col("unit_price_po"))
    
    # Define 'today' as the latest date from invoice_date or paid_date
    today = df.select(
        greatest(
            max("invoice_date"),
            max("paid_date")
        ).alias("today")
    ).first()["today"]
    
    # Create OnTimePayment column
    df = df.withColumn(
        "on_time_payment",
        when(
            col("paid_date").isNotNull() & (col("paid_date") <= col("due_date")),
            lit(True)
        ).when(
            col("paid_date").isNotNull() & (col("paid_date") > col("due_date")),
            lit(False)
        ).when(
            col("paid_date").isNull() & (col("due_date") <= lit(today)),
            lit(False)
        ).otherwise(lit(None)).cast(BooleanType())
    )
    
    # Create AgingDays column
    df = df.withColumn(
        "AgingDays",
        when(
            col("paid_date").isNull(),
            datediff(lit(today), col("due_date"))
        ).otherwise(
            datediff(col("paid_date"), col("invoice_date"))
        ).cast(IntegerType())
    )
    
    # Create Month column
    df = df.withColumn(
        "Month",
        date_format(col("invoice_date"), "yyyy-MM")
    )
    
    return df


# ============================================================================
# TABLE CREATION
# ============================================================================
# Execute the AP Invoices data transformation pipeline.

print(f"Loading source data from {INPUT_TABLE}...")
df = spark.table(INPUT_TABLE)

display(df)

In [0]:
# ============================================================================
# TABLE CREATION
# ============================================================================
# Execute the AP Invoices data transformation pipeline.

print(f"Loading source data from {INPUT_TABLE}...")
df = spark.table(INPUT_TABLE)

print("Standardizing date formats...")
df = standardize_dates(df, DATE_COLS)

print("Cleaning text and ID fields...")
df = clean_text_fields(df, TEXT_FIELDS, ID_FIELDS)

print("Enriching data with calculated columns...")
df = enrich_data(df)

print(f"Writing enriched data to {OUTPUT_TABLE}...")
df.write.format("delta").mode("overwrite").saveAsTable(OUTPUT_TABLE)

print("Transformation pipeline completed successfully!")


# Display results
display(df)

In [0]:

from pyspark.sql.functions import *


INPUT_TABLE = "ap.bronze.ap_invoices"
OUTPUT_TABLE = "ap.silver.ap_invoices_enriched"  # Adjust as needed
DATE_COLS = ["invoice_date", "due_date", "paid_date"]
TEXT_FIELDS = ["supplier", "category"]
ID_FIELDS = ["invoice_id", "cost_center", "poid"]


def standardize_dates(df: DataFrame, columns: list) -> DataFrame:
    """
    Convert date columns to date type safely.
    Tries multiple formats before falling back to default parsing.
    """
    common_formats = [
        "MM/dd/yyyy",  # 10/30/2025
        "MM/dd/yy",    # 10/30/25
        "dd/MM/yyyy",  # 30/10/2025
        "yyyy/MM/dd",  # 2025/10/30
        "yyyy-MM-dd"   # 2025-10-30
    ]
    
    for c in columns:
        # Try each format
        date_col = None
        for fmt in common_formats:
            temp_col = try_to_date(col(c), fmt)
            if date_col is None:
                date_col = temp_col
            else:
                # Coalesce to try next format if previous failed
                date_col = coalesce(date_col, temp_col)
        
        # Replace original column with parsed date
        df = df.withColumn(c, date_col)
    
    return df


print(f"Loading source data from {INPUT_TABLE}...")
df = spark.table(INPUT_TABLE)

print("Standardizing date formats...")
df = standardize_dates(df, DATE_COLS)
display(df)

### UDF approach

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime

def parse_date_flexible(date_str):
    """
    Parse date string with multiple format attempts.
    """
    if date_str is None or date_str.strip() == "":
        return None
    
    date_str = date_str.strip()
    
    # List of formats to try
    formats = [
        "%m/%d/%Y",  # 07/01/2025
        "%m/%d/%y",  # 07/01/25
        "%-m/%d/%Y", # 7/01/2025 (Unix-like systems)
        "%-m/%d/%y", # 7/01/25
        "%-m/%-d/%Y", # 7/1/2025
        "%-m/%-d/%y", # 7/1/25
        "%Y-%m-%d",  # 2025-07-01
        "%d/%m/%Y",  # 01/07/2025
    ]
    
    # Also try without leading zero removal (for Windows compatibility)
    import re
    # Remove leading zeros: "07/01/25" -> "7/1/25"
    normalized = re.sub(r'\b0(\d)', r'\1', date_str)
    
    for fmt in formats:
        for test_str in [date_str, normalized]:
            try:
                # Remove %-  for Windows (doesn't support it)
                fmt_clean = fmt.replace('%-', '%')
                return datetime.strptime(test_str, fmt_clean).date()
            except (ValueError, AttributeError):
                continue
    
    return None

# Register UDF
parse_date_udf = udf(parse_date_flexible, DateType())

def standardize_dates_udf(df, columns):
    """Use UDF for more flexible parsing"""
    for col_name in columns:
        if col_name in df.columns:
            df = df.withColumn(col_name, parse_date_udf(col(col_name)))
    return df

print(f"Loading source data from {INPUT_TABLE}...")
df = spark.table(INPUT_TABLE)

print("Standardizing date formats with UDF...")
df = standardize_dates_udf(df, DATE_COLS)

# Validation
for date_col in DATE_COLS:
    null_count = df.filter(col(date_col).isNull()).count()
    if null_count > 0:
        print(f"Warning: {null_count} nulls in {date_col}")

display(df)